In [ ]:
from pymongo import MongoClient

# ----------------------
client = MongoClient("mongodb://localhost:27017")

# ----------------------
print(client.list_database_names())


['admin', 'config', 'ejercicios', 'local', 'test']


In [3]:
db = client.ejercicios
movies_collection = db.MOVIES


## 0. ESTUDIO Y LIMPIEZA TABLA


### 0.1. Tipos de las columnas

In [4]:
import pandas as pd

df=pd.DataFrame(list(movies_collection.find()))
df.dtypes

_id                           object
plot                          object
genres                        object
runtime                      float64
cast                          object
num_mflix_comments           float64
title                         object
fullplot                      object
countries                     object
released              datetime64[ns]
directors                     object
rated                         object
awards                        object
lastupdated                   object
year                          object
imdb                          object
type                          object
tomatoes                      object
poster                        object
languages                     object
writers                       object
metacritic                   float64
dtype: object

### 0.2. Eliminación de columnas


In [5]:
movies_collection.update_many(
  {},  #no ponemos condición, queremos actualizar todos               
  { "$unset": { "fullplot": "" , "lastupdated": "" , "poster": "", "tomatoes": "", "writers": "", "metacritic": "", "num_mfilx_comments": ""} } 
)

UpdateResult({'n': 23539, 'nModified': 23539, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [6]:
df=pd.DataFrame(list(movies_collection.find()))
df.columns

Index(['_id', 'plot', 'genres', 'runtime', 'cast', 'num_mflix_comments',
       'title', 'countries', 'released', 'directors', 'rated', 'awards',
       'year', 'imdb', 'type', 'languages'],
      dtype='object')

## 1. FILTRADO DE LA TABLA

Debido a la gran cantidad de datos, vamos a filtrar nuestro db

### 1.1. Pelis recientes

Veamos primero el rango de años que tenemos

In [15]:
max_doc = movies_collection.find({}, {"_id": 0, "year": 1}).sort("year", -1).limit(1)
max_year = list(max_doc)[0]["year"]

min_doc = movies_collection.find({}, {"_id": 0, "year": 1}).sort("year", 1).limit(1)
min_year = list(min_doc)[0]["year"]

print(f'Rango: ({min_year}, {max_year})')


Rango: (1891, 2015è)


Vemos que hay algún tipo de error, por lo que vamos a limpiar los docs que no tengan year como un int

In [21]:
count_strings = movies_collection.count_documents({"year": {"$type": "string"}})
print(f"Documentos con year como string: {count_strings}")

movies_collection.delete_many({"year":{"$type":"string"}})

Documentos con year como string: 37


DeleteResult({'n': 37, 'ok': 1.0}, acknowledged=True)

Ahora sí, vemos el rango

In [22]:
max_doc = movies_collection.find({}, {"_id": 0, "year": 1}).sort("year", -1).limit(1)
max_year = list(max_doc)[0]["year"]

min_doc = movies_collection.find({}, {"_id": 0, "year": 1}).sort("year", 1).limit(1)
min_year = list(min_doc)[0]["year"]

print(f'Rango: ({min_year}, {max_year})')


Rango: (1891, 2016)


Nos quedaremos entonces sólo con las pelis posteriores a 1970

In [23]:
movies_collection.delete_many({"year":{"$lt":1970}})

DeleteResult({'n': 2674, 'ok': 1.0}, acknowledged=True)

### 1.2. Pelis por paises

Además, estudiaremos solo películas de los cinco países con más películas. Ahora bien, como countries es un array, tendremos que hacer un unwind de este campo

In [ ]:
query=[
  { "$unwind": "$countries" },       
  { "$group": { 
      "_id": "$countries", 
      "total": { "$sum": 1 } 
  } },  
  { "$sort": { "total": -1 } },
  { "$limit":5}
]

result = list(movies_collection.aggregate(query))
df_pelis_ppais=pd.DataFrame(result)
df_pelis_ppais

,_id,total
0,USA,10310
1,France,2778
2,UK,2592
3,Germany,1632
4,Canada,1333


Ahora sí, procedemos a eliminar aquellas que no tengan **ninguno** de los anteriores países dentro de su vector *countries*

In [25]:
entries_antes=movies_collection.count_documents({})

paises_permitidos= ['USA', 'France', 'UK', 'Germany', 'Canada']
eliminacion_paises={'countries': { "$elemMatch": { "$nin": paises_permitidos }}}
movies_collection.delete_many(eliminacion_paises)

entries_despues=movies_collection.count_documents({})

print(f'Documentos antes: {entries_antes}->Documentos despues: {entries_despues}')


Documentos antes: 20828->Documentos despues: 11915


## 2. ESTUDIO DE LOS DATOS

### 2.1. Cuál es la media de valoración de las pelis por país? Y la media de nominaciones a premios?

In [ ]:
query_pais=[
  { "$unwind": "$countries" },
  { "$match": {} },       
  { "$group": { 
      "_id": "$countries", 
      "media_nominaciones": { "$avg": "$awards.nominations"} , 
      "media_puntuacion":{"$avg": "$imdb.rating"} 
      } 
  },  
  { "$sort": { "media_nominaciones": -1 }}
]

results=list(movies_collection.aggregate(query_pais))
df_resultados_pais=pd.DataFrame(results)
df_resultados_pais

,_id,media_nominaciones,media_puntuacion
0,UK,7.409972,6.820944
1,Germany,5.792654,6.500474
2,USA,5.758137,6.439974
3,France,5.194093,6.700676
4,Canada,4.670175,6.320088


Por tanto, vemos que en ambos casos las pelis de UK son las que mas gustan tanto a nivel crítica cómo a nivel de público, por lo que produciremos nuestra peli allí

### 2.2. De qué género debería ser nuestra peli?

Para ello, vamos a mirar cuántas pelis de cada género hay en UK y su media de puntuación

In [40]:
query_generosUK=[
    {"$unwind": "$genres"}, 
    {"$match": {"countries" : 'UK'}},
    {"$group" : {"_id": "$genres", "total": {"$sum":1}, "media_puntuacion": {"$avg": "$imdb.rating"}}},
    {"$sort": {"media_puntuacion":-1}},
    {"$limit":5}
]

lista_docs=list(movies_collection.aggregate(query_generosUK))
df_generos=pd.DataFrame(lista_docs)
df_generos

,_id,total,media_puntuacion
0,Documentary,231,7.584848
1,News,2,7.550000
2,History,116,7.390517
3,Short,39,7.364872
4,War,65,7.223077


Ahora bien, este dato no sería del todo fiable, ya que si una película tiene una puntuación de 10 pero sólo ha sido valorada por una persona, este no es un dato representativo. Así, añadiremos un nuevo campo a imdb, el score, calculado como: 
$$
\text{Score} = \frac{v}{v + m} \cdot R + \frac{m}{v + m} \cdot C
$$

donde:

- \( R \) = puntuación media de la película  
- \( v \) = número de votos de la película  
- \( m \) = número mínimo de votos requeridos  
- \( C \) = media de todas las puntuaciones

Calculemos entonces m como el numero medio de votos de las pelis de UK y C la media de todas las puntuaciones


In [28]:
query_datos_imdb_UK=[
    {"$match": {"countries" : 'UK'}},
    {"$group" : {"_id": "UK", "media_votos": {"$avg": "$imdb.votes"},  "media_puntuaciones": {"$avg": "$imdb.rating"}}}
]

lista_datos_imdb_UK=list(movies_collection.aggregate(query_datos_imdb_UK))
df_datos_imdb_UK=pd.DataFrame(lista_datos_imdb_UK)
df_datos_imdb_UK

,_id,media_votos,media_puntuaciones
0,UK,35025.013333,6.820944


Ahora si, añadamos el key:value del score al objeto imdb y volvamos a hcer la consulta sobre promedio de score por géneros

In [29]:
m=34845
C=6.83

Convertimos en primer lugar los ratings y los votos a formato numero, rellenando los faltantes o los que nos den error con las medias

In [30]:
movies_collection.update_many(
    {"countries": "UK"},
    [
        {
            "$set": {
                "imdb.votes": {
                    "$convert": {
                        "input": "$imdb.votes",
                        "to": "int",
                        "onError": m,
                        "onNull": m
                    }
                },
                "imdb.rating": {
                    "$convert": {
                        "input": "$imdb.rating",
                        "to": "double",
                        "onError": C,
                        "onNull": C
                    }
                }
            }
        }
    ]
)


UpdateResult({'n': 1805, 'nModified': 184, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Ahora sí, añadimos el campo score

In [ ]:
movies_collection.update_many(
    {"countries": "UK"},
    [{"$set": {"imdb.score": {
                    "$add": [
                        {
                            "$multiply": [
                                {
                                    "$divide": [
                                        "$imdb.votes",
                                        {"$add": ["$imdb.votes", m]}
                                    ]
                                },
                                "$imdb.rating"
                            ]
                        },
                        {
                            "$multiply": [
                                {
                                    "$divide": [
                                        m,
                                        {"$add": ["$imdb.votes", m]}
                                    ]
                                },
                                C
                            ]
                        }
                    ]
                }
            }
        }
    ]
)

UpdateResult({'n': 1805, 'nModified': 1805, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Ahora sí, calculamos el **score** en lugar del rating por géneros para UK

In [32]:
query_generosUK=[
    {"$unwind": "$genres"}, 
    {"$match" : {"countries" : 'UK'}},
    {"$group" : {"_id": "$genres", "media_puntuacion": {"$avg": "$imdb.score"}}},
    {"$sort" : {"media_puntuacion":-1}},
    {"$limit" : 5}
]

lista_docs=list(movies_collection.aggregate(query_generosUK))
df_generos=pd.DataFrame(lista_docs)
df_generos

,_id,media_puntuacion
0,War,6.947473
1,History,6.941729
2,Sci-Fi,6.934417
3,Musical,6.921435
4,Biography,6.919719


Por lo tanto, concluimos que nuestra peli será de temática bélica e histórica. Veamos cuántas pelis cumplen con estos requisitos

### 2.3. Qué actores tendría que tener nuestra película?

Por ultimo, y para contentar ahora al equipo artístico (ya que la decisición del género de la película ha estado basada en los votos de IMDB), eligiremos a los actores de películas de War e History que hayan tenido mayor exitos en los premios

In [43]:
query_actores=[
    {"$unwind":"$cast"},
    {"$match":{"genres":{"$in":["War","History"]},"countries":"UK"}},
    {"$group":{"_id":"$cast","media_nominations":{"$avg":"$awards.nominations"},"media_wins":{"$avg":"$awards.wins"}}},
    {"$sort":{"media_wins":-1}}
]

df_actores=pd.DataFrame(list(movies_collection.aggregate(query_actores)))
df_actores

,_id,media_nominations,media_wins
0,Bryan Batt,256.0,267.0
1,Dwight Henry,256.0,267.0
2,Dickie Gravois,256.0,267.0
3,Chiwetel Ejiofor,133.5,133.5
4,Kristin Scott Thomas,53.0,68.0
...,...,...,...
529,Jodie Whittaker,2.0,0.0
530,George Cooper,1.0,0.0
531,Francis Magee,1.0,0.0
532,Ron Cook,2.0,0.0


In [44]:
query_actores=[
    {"$unwind":"$cast"},
    {"$match":{"genres":{"$in":["War","History"]},"countries":"UK","$expr": {"$gte": ["$awards.nominations", "$awards.wins"]}}},
    {"$group":{"_id":"$cast","media_nominations":{"$avg":"$awards.nominations"},"media_wins":{"$avg":"$awards.wins"}}},
    {"$sort":{"media_wins":-1}}
]

df_actores=pd.DataFrame(list(movies_collection.aggregate(query_actores)))
df_actores

,_id,media_nominations,media_wins
0,David Oyelowo,73.0,55.0
1,Jim France,73.0,55.0
2,Trinity Simone,73.0,55.0
3,Carmen Ejogo,73.0,55.0
4,Geoffrey Rush,48.0,37.0
...,...,...,...
371,Juliette Binoche,1.0,0.0
372,Natalie Portman,3.0,0.0
373,Scarlett Johansson,3.0,0.0
374,Dominic West,3.0,0.0
